In [1]:
import torch
import torch.nn as nn 
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

cpu


In [2]:
with open('Frankenstein.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'æ', 'è', 'é', 'ê', 'ô', '—', '‘', '’', '“', '”', '\ufeff']


In [3]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)


In [4]:
# Define percentage
n = int(0.8*len(data))
# 80 % split for training 
train_data = data[:n]
# 20 % split for validating
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x =  torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x,y
    
x,y = get_batch('train')
print('inputs:')

print(x)
print('targets:')
print(y)


inputs:
tensor([[51, 59, 47, 55, 60, 50, 51, 64],
        [69,  1, 54, 51, 47, 68, 55, 58],
        [55, 65, 66, 67, 64, 48,  1, 66],
        [ 1, 59, 55, 65, 66, 64, 51, 65]])
targets:
tensor([[59, 47, 55, 60, 50, 51, 64,  1],
        [ 1, 54, 51, 47, 68, 55, 58, 71],
        [65, 66, 67, 64, 48,  1, 66, 54],
        [59, 55, 65, 66, 64, 51, 65, 65]])


In [5]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is', context, 'target is', target)

when input is tensor([83]) target is tensor(26)
when input is tensor([83, 26]) target is tensor(64)
when input is tensor([83, 26, 64]) target is tensor(47)
when input is tensor([83, 26, 64, 47]) target is tensor(60)
when input is tensor([83, 26, 64, 47, 60]) target is tensor(57)
when input is tensor([83, 26, 64, 47, 60, 57]) target is tensor(51)
when input is tensor([83, 26, 64, 47, 60, 57, 51]) target is tensor(60)
when input is tensor([83, 26, 64, 47, 60, 57, 51, 60]) target is tensor(65)


In [6]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [7]:
# Define a class that accepts a neural network module for learning
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # Create a table with the dimensions of the vocab size each token is then mapped in the embedding vector
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    # Define forward pass of the neural network. Two parameters are taken index (input tokens) and targets (ground truth tokens: optional)
    def forward(self, index, targets=None):
        # Input is passed to table for init
        logits = self.token_embedding_table(index)
        if targets is None:
            loss = None
        else:
            # Cross entropy is calculated across the predicted logits and true targets
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss

    # Define a function to generate new tokens given a starting context
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        # new tokens generated in loop appending to index tensor
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:,-1,:] # becomes (B, C)
            # Apply softmax to get probabilities 
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distrubution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # Append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)
# Starting tensor created to represent padding or starting token
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


H[(iU_W0èRpW5éS9R’Gag’fTOpLN9KH
’
vUn“-DI,’ê,205é“ æA- Ru;;ôiFèE,_é!—UAmo4rj(UérFqêY6?YyLw“qRètAH‘H[æedavJSBkAG9Rj)7OjJUou],2Jp4ôS B2v4R0“beB:!;ENhc?(xqtxg9_z”’tDF9NsM2z_IqôA﻿uBWê6?2it.æjSO wIqa‘sedyJ],7BIq:;0‘RsMsS?LGét01]yoUMBRj﻿zdFèu4rFèwNéJ“a1.-[]j-FSnwseRi,CkL4V6j“:6D)6éI.7)è1UA37bêôzax?D!6p3YIDRtu”Y﻿’
NNWlr)MPpPh5HR!kLBeS’hRpMyUK‘9D(p1fqeUlmE3 n3q1﻿mEôSo45jVL,wgê6cPez0Ie12hGdlê)1’“‘Rp2.5LVtléf_ôHYCièMn” 
pV“bYIz]ès:[(jGmbô”?g;OpR6?EhBv“!6.Aq8‘f6ôê(”cwc[!’tgTx?”8P8L4r”ê3B(”V(.p‘L(éeê﻿;D71èL


In [11]:
# Create a PyTorch Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f} ")
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss 
    logits, loss = model.forward(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.440, val loss: 4.424 
step: 250, train loss: 4.363, val loss: 4.365 
step: 500, train loss: 4.337, val loss: 4.293 
step: 750, train loss: 4.280, val loss: 4.253 
4.042873382568359


In [9]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


MF‘2h?YJ94?KJwé:dvôS9(WO(b85VfD(q.5H4L,0gkBiVtJ—zé-T_’IæW49BPG]OO(v08w‘2h94q9CHudP?MaUy?ô“)pv[q9[j2i-[Y2FdSbé “8tHiLt_WPJ‘SEySè” qF”Lér9RkiGw[ô,f-[æt[4, tUU_65tx?D?Avn3qê,eK2f6tilD!3J7OK:68!WPLx”’é .[yTDx
é_8T_OFèk68

PCêki TtA,!,hê)(b’è6GIqz:Gér‘gzAkc[]é3I
fgMJdFèsyD; 8m1VAuqWl﻿hRê_W]g18UA3Rk;oyLRsy,
S0d9(cySLNhLYGcEôUM,héWu—KJW]JHNU.tJé éf_Ou_OG t3é8mx2]FèbD?Mv:‘éyêæzaIz:[CU77HRT﻿2w8

iéGa‘Mu-V(æAAAGbhoif27wJ1IYR7!3ID8m[oyrgh4PwC9éViédu9ALmkSd,],hg!ô3NJ1SludaaJêHô]RW“I3U4P4r1M—ê9j;(A,B
xyts!3b
